# Lab | Hypothesis Testing

**Objective**

Welcome to the Hypothesis Testing Lab, where we embark on an enlightening journey through the realm of statistical decision-making! In this laboratory, we delve into various scenarios, applying the powerful tools of hypothesis testing to scrutinize and interpret data.

From testing the mean of a single sample (One Sample T-Test), to investigating differences between independent groups (Two Sample T-Test), and exploring relationships within dependent samples (Paired Sample T-Test), our exploration knows no bounds. Furthermore, we'll venture into the realm of Analysis of Variance (ANOVA), unraveling the complexities of comparing means across multiple groups.

So, grab your statistical tools, prepare your hypotheses, and let's embark on this fascinating journey of exploration and discovery in the world of hypothesis testing!

**Challenge 1**

In this challenge, we will be working with pokemon data. The data can be found here:

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/pokemon.csv

In [1]:
#libraries
import pandas as pd
import scipy.stats as st
import numpy as np



In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/pokemon.csv")


- We posit that Pokemons of type Dragon have, on average, more HP stats than Grass. Choose the propper test and, with 5% significance, comment your findings.

In [5]:
# Filter data for Dragon-type and Grass-type Pokémon
dragon_hp = df[df['Type 1'] == 'Dragon']['HP']
grass_hp = df[df['Type 1'] == 'Grass']['HP']

# Perform a one-sided two-sample t-test
t_stat, p_value = st.ttest_ind(dragon_hp, grass_hp, alternative='greater', nan_policy='omit')

# Print the result
print(f"T-statistic: {t_stat:.3f}, P-value: {p_value:.3f}")

T-statistic: 3.590, P-value: 0.000


- We posit that Legendary Pokemons have different stats (HP, Attack, Defense, Sp.Atk, Sp.Def, Speed) when comparing with Non-Legendary. Choose the propper test and, with 5% significance, comment your findings.


In [6]:
# Separate Legendary and Non-Legendary Pokémon
legendary_stats = df[df['Legendary'] == True][['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']]
non_legendary_stats = df[df['Legendary'] == False][['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']]

# Conduct a two-sample t-test for each stat
results = {}
for stat in legendary_stats.columns:
    t_stat, p_value = st.ttest_ind(legendary_stats[stat], non_legendary_stats[stat], nan_policy='omit')
    results[stat] = (t_stat, p_value)

# Print the results
for stat, (t_stat, p_value) in results.items():
    print(f"{stat}: T-statistic = {t_stat:.3f}, P-value = {p_value:.3f}")

HP: T-statistic = 8.036, P-value = 0.000
Attack: T-statistic = 10.397, P-value = 0.000
Defense: T-statistic = 7.181, P-value = 0.000
Sp. Atk: T-statistic = 14.191, P-value = 0.000
Sp. Def: T-statistic = 11.038, P-value = 0.000
Speed: T-statistic = 9.765, P-value = 0.000


**Challenge 2**

In this challenge, we will be working with california-housing data. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/california_housing.csv

In [7]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/california_housing.csv")
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


**We posit that houses close to either a school or a hospital are more expensive.**

- School coordinates (-118, 37)
- Hospital coordinates (-122, 34)

We consider a house (neighborhood) to be close to a school or hospital if the distance is lower than 0.50.

Hint:
- Write a function to calculate euclidean distance from each house (neighborhood) to the school and to the hospital.
- Divide your dataset into houses close and far from either a hospital or school.
- Choose the propper test and, with 5% significance, comment your findings.
 

In [8]:
# Define the coordinates for the school and hospital
school_coords = (-118, 37)
hospital_coords = (-122, 34)

# Define a function to calculate Euclidean distance
def euclidean_distance(row, coords):
    return np.sqrt((row['longitude'] - coords[0])**2 + (row['latitude'] - coords[1])**2)

# Calculate distance to school and hospital
df['dist_school'] = df.apply(euclidean_distance, coords=school_coords, axis=1)
df['dist_hospital'] = df.apply(euclidean_distance, coords=hospital_coords, axis=1)

# Create a binary variable indicating if the house is close to either school or hospital
df['close_to_amenities'] = ((df['dist_school'] < 0.50) | (df['dist_hospital'] < 0.50)).astype(int)

# Separate data into houses close and far
close = df[df['close_to_amenities'] == 1]['median_house_value']
far = df[df['close_to_amenities'] == 0]['median_house_value']

# Perform a two-sample t-test (or Mann-Whitney U test if not normal)
t_stat, p_value = st.ttest_ind(close, far, nan_policy='omit')

# Print the result
print(f"T-statistic: {t_stat:.3f}, P-value: {p_value:.3f}")

T-statistic: -2.215, P-value: 0.027
